In [3]:
import numpy as np
import pandas as pd
import dask.dataframe as dk
import tensorflow as tf
input_file = "npm_features_dataset.csv"
output_file =  "npm_shuffled_1.csv"

df = pd.read_csv(input_file)
shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)
shuffled_df.to_csv(output_file, index=False)

print(f"Đã xáo trộn và lưu file thành công tại: {output_file}")

Đã xáo trộn và lưu file thành công tại: npm_shuffled_1.csv
